In [2]:
from data_loader import load_data
import random

In [3]:
def random_feasible_solution(n_depots, n_customers, depot_capacities, customer_demands, costs):
    while True:
        assignment = [-1] * n_customers
        remaining_capacity = depot_capacities[:]
        for i in range(n_customers):
            shuffled_depots = list(range(n_depots))
            random.shuffle(shuffled_depots)
            for d in shuffled_depots:
                if remaining_capacity[d] >= customer_demands[i]:
                    assignment[i] = d
                    remaining_capacity[d] -= customer_demands[i]
                    break
        if -1 not in assignment:
            open_depots = list(set(assignment))
            return {'assignment': assignment, 'open_depots': open_depots}


In [6]:
def total_cost(solution, setup_costs, costs):
    depot_cost = sum(setup_costs[d] for d in solution['open_depots'])
    transport_cost = sum(costs[i][solution['assignment'][i]] for i in range(len(solution['assignment'])))
    return depot_cost + transport_cost

def delta_cost(old_solution, new_solution, setup_costs, costs):
    old_open = set(old_solution['open_depots'])
    new_open = set(new_solution['open_depots'])
    depot_diff = sum(setup_costs[d] for d in new_open - old_open) - sum(setup_costs[d] for d in old_open - new_open)
    assignment_old = old_solution['assignment']
    assignment_new = new_solution['assignment']
    transport_diff = sum(
        costs[i][assignment_new[i]] - costs[i][assignment_old[i]]
        for i in range(len(assignment_old)) if assignment_new[i] != assignment_old[i]
    )
    return depot_diff + transport_diff

def get_neighbors_limited(solution, n_depots, depot_capacities, customer_demands, k=3):
    neighbors = []
    assignment = solution['assignment']
    for _ in range(k):
        i = random.randint(0, len(assignment)-1)
        current_d = assignment[i]
        for d in range(n_depots):
            if d != current_d:
                new_assignment = assignment[:]
                new_assignment[i] = d
                if is_feasible(new_assignment, depot_capacities, customer_demands, n_depots):
                    open_depots = list(set(new_assignment))
                    neighbors.append({'assignment': new_assignment, 'open_depots': open_depots})
    return neighbors
def is_feasible(assignment, depot_capacities, customer_demands, n_depots):
    depot_loads = [0] * n_depots
    for i, d in enumerate(assignment):
        depot_loads[d] += customer_demands[i]
    return all(depot_loads[i] <= depot_capacities[i] for i in range(n_depots))

In [7]:
def tabu_search(n_depots, n_customers, depot_capacities, customer_demands, 
                setup_costs, costs, max_iter=100, tabu_tenure=7, neighbor_k=10):
    current = random_feasible_solution(n_depots, n_customers, depot_capacities, customer_demands, costs)
    best = current
    best_cost = total_cost(current, setup_costs, costs)
    tabu_list = []

    for _ in range(max_iter):
        neighbors = get_neighbors_limited(current, n_depots, depot_capacities, customer_demands, k=neighbor_k)
        filtered = []
        for n in neighbors:
            assignment_tuple = tuple(n['assignment'])
            if assignment_tuple not in tabu_list or total_cost(n, setup_costs, costs) < best_cost:
                filtered.append(n)
        if not filtered:
            break
        candidate = min(filtered, key=lambda s: total_cost(s, setup_costs, costs))
        cost_candidate = total_cost(candidate, setup_costs, costs)

        if cost_candidate < best_cost:
            best = candidate
            best_cost = cost_candidate

        tabu_list.append(tuple(candidate['assignment']))
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)

        current = candidate

    return best, best_cost


In [22]:
def simulated_annealing(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs,
                        initial_temp=1000, cooling_rate=0.95, max_iter=100, neighbor_k=5):

    # ✨ Tabu Search ile başla
    initial_solution, _ = tabu_search(n_depots, n_customers, depot_capacities, customer_demands, setup_costs, costs)

    current = initial_solution
    best = current
    temp = initial_temp

    for _ in range(max_iter):
        neighbors = get_neighbors_limited(current, n_depots, depot_capacities, customer_demands, k=neighbor_k)
        if not neighbors:
            break
        candidate = random.choice(neighbors)
        cost_diff = total_cost(candidate, setup_costs, costs) - total_cost(current, setup_costs, costs)
        if cost_diff < 0 or random.random() < np.exp(-cost_diff / temp):
            current = candidate
            if total_cost(current, setup_costs, costs) < total_cost(best, setup_costs, costs):
                best = current
        temp *= cooling_rate

    best_cost = total_cost(best, setup_costs, costs)
    return best, best_cost


In [12]:
import numpy as np

In [ ]:
filename = 'Dataset/wl_25'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)


assert len(depot_capacities) == n_depots, "depot_capacities boyutu yanlış"
assert all(len(row) == n_depots for row in costs), "costs matrisi sütun sayısı hatalı"
assert len(customer_demands) == n_customers, "customer_demands boyutu yanlış"

# Depo ve müşteri sayısını tekrar belirleyelim
n_depots = len(depot_capacities)  # Depo sayısı
n_customers = len(customer_demands)  # Müşteri sayısı

# costs matrisini numpy dizisine çevir
costs = np.array(costs)  # Müşteri-depo maliyetleri

# costs matrisinin boyutlarını kontrol et
print(f"Mevcut costs boyutu: {costs.shape}")
print(f"Beklenen costs boyutu: ({n_depots}, {n_customers})")

# costs matrisinin boyutları uyumsuzsa, uygun şekilde düzeltelim
if costs.shape[0] != n_depots or costs.shape[1] != n_customers:
    print("costs matrisinin boyutları uyumsuz!")
    print(f"Mevcut costs boyutu: {costs.shape}")
    print(f"Beklenen costs boyutu: ({n_depots}, {n_customers})")

    # costs matrisinin boyutlarını transpoze edelim
    if costs.shape[0] == n_customers and costs.shape[1] == n_depots:
        costs = costs.T  # Eğer satırlar ve sütunlar yer değişmişse, transpoz alıyoruz
        print("costs matrisini transpoze ettik.")
    elif costs.shape[0] != n_depots or costs.shape[1] != n_customers:
        print("costs matrisinin boyutları hala uyumsuz!")
        costs = costs[:n_depots, :n_customers]
        print("costs matrisini uygun boyutlarda kestik.")
    
    print(f"Yeni costs boyutu: {costs.shape}")
else:
    print("costs matrisinin boyutları uyumlu.")

# 2. SA ile bu çözüm üzerinden daha da iyisini aramaya çalış
result, cost = simulated_annealing(n_depots, n_customers, depot_capacities, customer_demands, setup_costs, costs)
print("Best solution found by SA:", result)
print("Cost of the best solution:", cost)

Mevcut costs boyutu: (50, 25)
Beklenen costs boyutu: (25, 50)
costs matrisinin boyutları uyumsuz!
Mevcut costs boyutu: (50, 25)
Beklenen costs boyutu: (25, 50)
costs matrisini transpoze ettik.
Yeni costs boyutu: (25, 50)
Depot capacities length: 25
Costs matrix dimensions: 25 x 50
Customer demands length: 50
